In [ ]:
# This notebook is used to create the PERSON table
# Based on the following documentations: 
#   https://ohdsi.github.io/CommonDataModel/cdm53.html#person
#   https://documentation-snds.health-data-hub.fr/omop/documentation_etl/person.html

import pandas as pd
import hashlib

# Loading data
df_raw = pd.read_csv("../data/raw/ir_ben_r.csv", sep=",", dtype=str)

df_raw


,NUM_ENQ,ben_sex_cod,ben_nai_ann,ben_nai_moi,ben_res_dpt,ben_res_reg
0,DPXX:00000000000000001X,2,1963,12,75,114
1,DPXX:000000000000002X,1,1971,2,93,114
2,DPXX:000000000000003X,1,1962,12,93,114
3,DPXX:000000000000004X,2,1959,3,94,114
4,DPXX:000000000000005X,1,1998,4,93,114
5,DPXX:000000000000006X,1,1958,11,75,116
6,DPXX:000000000000007X,2,1957,12,75,114
7,DPXX:000000000000008X,1,1964,11,93,116
8,DPXX:000000000000009X,1,1988,7,75,115
9,DPXX:000000000000010X,2,1965,3,94,115


In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   NUM_ENQ      20 non-null     object
 1   ben_sex_cod  20 non-null     object
 2   ben_nai_ann  20 non-null     object
 3   ben_nai_moi  20 non-null     object
 4   ben_res_dpt  20 non-null     object
 5   ben_res_reg  20 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB


In [9]:
# Compute length of each value
df_raw['NUM_ENQ_length'] = df_raw['NUM_ENQ'].str.len()

# Find the most frequent length
target_length = df_raw['NUM_ENQ_length'].mode()[0]

# Function to normalize length
def normalize_id(value, target_length):
    if len(value) == target_length:
        return value
    prefix, suffix = value.split(":")
    # Remove zeros
    trimmed_suffix = suffix.lstrip("0")
    # Add enough zeros to reach most frequent length
    new_suffix = trimmed_suffix.zfill(target_length - len(prefix) - 1)
    return f"{prefix}:{new_suffix}"

# Use function
df_raw['NUM_ENQ'] = df_raw['NUM_ENQ'].apply(lambda x: normalize_id(x, target_length))

# Remove the temporary column
df_raw.drop(columns=['NUM_ENQ_length'], inplace=True)

df_raw

,NUM_ENQ,ben_sex_cod,ben_nai_ann,ben_nai_moi,ben_res_dpt,ben_res_reg
0,DPXX:000000000000001X,2,1963,12,75,114
1,DPXX:000000000000002X,1,1971,2,93,114
2,DPXX:000000000000003X,1,1962,12,93,114
3,DPXX:000000000000004X,2,1959,3,94,114
4,DPXX:000000000000005X,1,1998,4,93,114
5,DPXX:000000000000006X,1,1958,11,75,116
6,DPXX:000000000000007X,2,1957,12,75,114
7,DPXX:000000000000008X,1,1964,11,93,116
8,DPXX:000000000000009X,1,1988,7,75,115
9,DPXX:000000000000010X,2,1965,3,94,115


In [14]:
# Mapping of gender_concept_id
gender_map = {
    "1": 8507,  # Homme
    "2": 8532   # Femme
}

def hash_to_int(value):
    # Use SHA-256
    hash_obj = hashlib.sha256(str(value).encode())
    return int(hash_obj.hexdigest(), 16) % (2**31 - 1)

# Apply hash_to_int function
df_raw["person_id"] = df_raw["NUM_ENQ"].apply(hash_to_int).astype("int32")

# Apply mapping
df_raw["gender_concept_id"] = df_raw["ben_sex_cod"].map(gender_map).astype("int32")
df_raw["year_of_birth"] = df_raw["ben_nai_ann"].astype("int32")
df_raw["month_of_birth"] = df_raw["ben_nai_moi"].astype("int32")
df_raw["location_id"] = df_raw["ben_res_dpt"] + df_raw["ben_res_reg"]

# Keep and rename mandatory columns
df_processed = df_raw[[
    "person_id",
    "gender_concept_id",
    "year_of_birth",
    "month_of_birth",
    "NUM_ENQ",
    "location_id",
    "ben_sex_cod"
]].rename(columns={
    "NUM_ENQ": "person_source_value",
    "ben_sex_cod": "gender_source_value"
})

# Update datatype to match SNDS needs 
df_processed = df_processed.astype({
    "person_source_value": "string",
    "location_id": "int32",
    "gender_source_value": "string",
})

df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   person_id            20 non-null     int32 
 1   gender_concept_id    20 non-null     int32 
 2   year_of_birth        20 non-null     int32 
 3   month_of_birth       20 non-null     int32 
 4   person_source_value  20 non-null     string
 5   location_id          20 non-null     int32 
 6   gender_source_value  20 non-null     string
dtypes: int32(5), string(2)
memory usage: 852.0 bytes


In [11]:
df_processed

,person_id,gender_concept_id,year_of_birth,month_of_birth,person_source_value,location_id,gender_source_value
0,346808875,8532,1963,12,DPXX:000000000000001X,75114,2
1,689659788,8507,1971,2,DPXX:000000000000002X,93114,1
2,1279245282,8507,1962,12,DPXX:000000000000003X,93114,1
3,1293505698,8532,1959,3,DPXX:000000000000004X,94114,2
4,1379382330,8507,1998,4,DPXX:000000000000005X,93114,1
5,1828535360,8507,1958,11,DPXX:000000000000006X,75116,1
6,1734146002,8532,1957,12,DPXX:000000000000007X,75114,2
7,74159434,8507,1964,11,DPXX:000000000000008X,93116,1
8,1631139980,8507,1988,7,DPXX:000000000000009X,75115,1
9,764181146,8532,1965,3,DPXX:000000000000010X,94115,2


In [12]:
# Save as CSV
df_processed.to_csv("../data/processed/PERSON.csv", index=False)